In [1]:
import pandas as pd
import numpy as np
import sidetable
import datetime as dt

df1 = pd.read_csv('Advent_B2B_Data_Work_Experience.csv')
df2 = pd.read_csv('Advent_B2B_Data_Edu_Experience.csv')
df = pd.merge(df1, df2, how="outer", on=["id"])

In [2]:
# Assumably, those without end day is still working in the existing company. Then, the end day is 1/4/2021
df['end_day'] = df['end_day'].fillna('1/4/2021')
df['start_day'] = pd.to_datetime(df['start_day'])
df['end_day'] = pd.to_datetime(df['end_day'])
# Assumably, those without fist job start started to work in the existing company. Then, the first job start is the start_day
df['first_job_start'] = df['first_job_start'].fillna(df['start_day'])
df['first_job_start']= pd.to_datetime(df['first_job_start'])
df['left_after_2020'] = np.where(df['end_day'].dt.year > 2020, True, False)
df['join_from_2015'] = np.where(df['start_day'].dt.year <= 2015, True, False)
df['existing_work_years'] = df['end_day'].dt.year - df['start_day'].dt.year
df['total_working_days'] = (df['end_day'] - df['first_job_start'])/np.timedelta64(1,'D')
df['start_year'] = df['start_day'].dt.year
df['end_year'] = df['end_day'].dt.year

In [3]:
df.head()

,id,headline_location,company,exp_location,title,description,start_day,end_day,first_job_start,number_of_recommendations,school,degree,time_start,time_end,left_after_2020,join_from_2015,existing_work_years,total_working_days,start_year,end_year
0,2ff517bd-3566-481e-a289-52ce4fb4ae2e,"Aliso Viejo, California, United States",Company_N,Greater Minneapolis-St. Paul Area,Spine Specialist,"Company_N, Inc., is a medical device company f...",2012-11-01,2015-03-01,2004-10-01,5.3,University of Minnesota,"Bachelor of Arts, Journalism",1997,2002,False,True,3,3803.0,2012,2015
1,78a0c7be-de6e-4af6-bab7-050a436e19fe,Singapore,Company_N,NaN,"Associate Manager, SEA Market Development",Managed the product throughout the product lif...,2010-07-01,2012-05-01,2005-09-01,3.6,Marketing Institute of Singapore,Graduate Diploma in Marketing,2008,2008,False,True,2,2434.0,2010,2012
2,78a0c7be-de6e-4af6-bab7-050a436e19fe,Singapore,Company_N,NaN,"Associate Manager, SEA Market Development",Managed the product throughout the product lif...,2010-07-01,2012-05-01,2005-09-01,3.6,National University of Singapore,"Bachelor of Engineering (B.Eng.), Electrical a...",2001,2005,False,True,2,2434.0,2010,2012
3,78a0c7be-de6e-4af6-bab7-050a436e19fe,Singapore,Company_N,NaN,Senior Spine Specialist,Discussed clinically and provided technical su...,2009-12-01,2010-07-01,2005-09-01,3.6,Marketing Institute of Singapore,Graduate Diploma in Marketing,2008,2008,False,True,1,1764.0,2009,2010
4,78a0c7be-de6e-4af6-bab7-050a436e19fe,Singapore,Company_N,NaN,Senior Spine Specialist,Discussed clinically and provided technical su...,2009-12-01,2010-07-01,2005-09-01,3.6,National University of Singapore,"Bachelor of Engineering (B.Eng.), Electrical a...",2001,2005,False,True,1,1764.0,2009,2010


In [4]:
df.stb.missing(style=True)

,missing,total,percent
description,21709,"32,952",65.88%
time_end,9076,"32,952",27.54%
exp_location,8201,"32,952",24.89%
time_start,5661,"32,952",17.18%
degree,4754,"32,952",14.43%
school,893,"32,952",2.71%
number_of_recommendations,12,"32,952",0.04%
start_year,0,"32,952",0.00%
total_working_days,0,"32,952",0.00%
existing_work_years,0,"32,952",0.00%


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32952 entries, 0 to 32951
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         32952 non-null  object        
 1   headline_location          32952 non-null  object        
 2   company                    32952 non-null  object        
 3   exp_location               24751 non-null  object        
 4   title                      32952 non-null  object        
 5   description                11243 non-null  object        
 6   start_day                  32952 non-null  datetime64[ns]
 7   end_day                    32952 non-null  datetime64[ns]
 8   first_job_start            32952 non-null  datetime64[ns]
 9   number_of_recommendations  32940 non-null  float64       
 10  school                     32059 non-null  object        
 11  degree                     28198 non-null  object        
 12  time

In [6]:
com_h = df[df.company == 'Company_H']
com_n = df[df.company == 'Company_N']
com_a = df[df.company == 'Company_A']

### 2.	Calculate full-time employee growth for the 3 companies from 2015 to 2020 (annually)

In [7]:
years = range(2014, 2021)
net_growth = list()
for y in years:
    net_growth.append(y)
    net_growth.append(com_a[com_a['start_year'] == y].count()['id'] - com_a[com_a['end_year'] == y].count()['id'])
    net_growth.append(com_h[com_h['start_year'] == y].count()['id'] - com_h[com_h['end_year'] == y].count()['id'])
    net_growth.append(com_n[com_n['start_year'] == y].count()['id'] - com_n[com_n['end_year'] == y].count()['id'])
net_growth = np.array(net_growth).reshape(len(years), -1)
print(net_growth.shape[0])

7


In [8]:
net_growth = pd.DataFrame(data=net_growth, columns=['Years', 'Company A', 'Company H', 'Company N'])
net_growth.set_index('Years', inplace=True)

In [9]:
# The first row is NaN because there is no previous term to compare. The second row is the percentage change between 2015 and 2014. -0.080189 means there was a tiny loss growth in 2015 against 2014.
net_growth.pct_change()

,Company A,Company H,Company N
Years,,,
2014,NaN,NaN,NaN
2015,-0.080189,1.468750,0.321905
2016,1.200000,0.253165,0.556196
2017,-0.473193,2.774411,-0.487963
2018,0.371681,-0.783229,0.088608
2019,0.038710,-0.152263,0.265781
2020,0.282609,0.300971,-1.018373


#### a.	Which company is growing the fastest? 

In [31]:
# This problem can be understood not only either in absolute number or percentage. To simplify it, I sum up all percentage change
net_growth.pct_change().sum()

Company A    1.34
Company H    3.86
Company N   -0.27
dtype: float64

It can be seen that Company H is growing the fastest

### Quetion 3: What is the most common job type for each of the 3 companies? Please define your methodology and how you approached this question

I spend about 10 minutes to scan all titles in the file. Briefly, I can classify them into 6 groups, but it's not the best. It's better to have descriptions of all titles (no missing values) that I can fit into some Natural Language Processing model.

In [41]:
import re
for post in [com_a.title.unique(), com_h.title.unique(), com_n.title.unique()]:
    g1 = re.findall(b'[Tt]echnician | [Aa]nalyst | [Pp]rogrammer | [Ee]ngineer', post)
    g2 = re.findall(b'[Ss]pecialist | [Cc]onsutant', post)
    g3 = re.findall(b'[Ii]ntern | [Ii]nternship', post)
    g4 = re.findall(b'[Ss]enior | [Ll]ead | [Ll]eader | [Aa]dministration | [Aa]dministrator | [Mm]anager', post)
    g5 = re.findall(b'[Aa]ssociate | [Cc]oordinator | [Cc]lerk', post)
    g6 = re.findall(b'[Dd]irector | [Vv][Pp] | [Ee][Vv][Pp] | [Vv]ice [Pp]resident', post)
    big_g = [g1, g2, g3, g4, g5, g6]
    for g in big_g:
        print(g)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [34]:
for i in df.title.unique():
    print(i)

Spine Specialist
Associate Manager, SEA Market Development
Senior Spine Specialist
Leader Business Transformation Office Global Operations
Senior Director Clinical Operations
Director Of Operations NCS
Senior Accounts Payable Lead
Sales Director-East
Algorithm Developer
Medical Sales - Spinal Implants
Sales Coordinator
Senior Administrative Assistant
Business Systems Analyst
Sr CNC Programmer
Pricing Analyst
Human Resources
Spine Associate
Manufacturing Consultant
R&D Intern
Associate Intraoperative Neurophysiologist
Quality Assurance Associate
Instrument Technician
Desktop Support Specialist
Spine specialist
Director, NCS IT and Infrastructure
Senior Manufacturing Engineer
Manufacturing Engineer
Sales Associate
Sr. Associate Product Manager, X360
Sr. Associate Product Manager, Fixation Procedures
Associate Manufacturing Manager
Senior Accountant
Senior SAP Basis/Security Administrator
Sr. product manager procedures Europe
Lead Project Manager
Global Logistics Manager
Associate Product

### Question 4: Which city has the most full-time employees for each of the 3 companies? 

In [10]:
print('The city with the most full-time employees for company_A is ' + str(com_a['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_a['headline_location'].value_counts()[0])) + ' employees')
print('The city with the most full-time employees for company_H is ' + str(com_h['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_h['headline_location'].value_counts()[0])) + ' employees')
print('The city with the most full-time employees for company_N is ' + str(com_n['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_n['headline_location'].value_counts()[0])) + ' employees')

The city with the most full-time employees for company_A is Luxembourg with 1377 employees
The city with the most full-time employees for company_H is United Arab Emirates with 761 employees
The city with the most full-time employees for company_N is San Diego, California, United States with 6627 employees


Due to the different formats in names of locations, I ouput the whole names of them

### Question 5: From 2015 to 2020, how many employees left their firm every year across the 3 firms?
#### a.	How would you standardize this metric for comparison’s sake?

In [11]:
com_a_left_2015_2020 = com_a[(com_a['join_from_2015'] == True) & (com_a['left_after_2020'] == False)].reset_index()
com_h_left_2015_2020 = com_h[(com_h['join_from_2015'] == True) & (com_h['left_after_2020'] == False)].reset_index()
com_n_left_2015_2020 = com_n[(com_a['join_from_2015'] == True) & (com_n['left_after_2020'] == False)].reset_index()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
com_a_left_2015_2020.count()

index                        1042
id                           1042
headline_location            1042
company                      1042
exp_location                  769
title                        1042
description                   394
start_day                    1042
end_day                      1042
first_job_start              1042
number_of_recommendations    1042
school                        988
degree                        819
time_start                    895
time_end                      737
left_after_2020              1042
join_from_2015               1042
existing_work_years          1042
total_working_days           1042
start_year                   1042
end_year                     1042
dtype: int64

The count numbers are not uniform among columns because there are missing values in some of them (exp_location, description, school, etc.). So, I take the number of index count because index cannot be missing. Then I divide it by 5 to get the average (every year).

In [13]:
print('The average number of employees leaving company A from 2015 to 2020 is ' + str(float(max(com_a_left_2015_2020.count()) / 5)))
print('The average number of employees leaving company H from 2015 to 2020 is ' + str(float(max(com_h_left_2015_2020.count()) / 5)))
print('The average number of employees leaving company N from 2015 to 2020 is ' + str(float(max(com_n_left_2015_2020.count()) / 5)))

The average number of employees leaving company A from 2015 to 2020 is 208.4
The average number of employees leaving company H from 2015 to 2020 is 86.6
The average number of employees leaving company N from 2015 to 2020 is 0.0


I will standardize them by this formular: X_standardized = (X - mean) / standard_deviation

In [14]:
numbers_of_leaving_from_2015_to_2020 = [208.4, 86.6, 0]
standardized = []
mean = np.mean(numbers_of_leaving_from_2015_to_2020)
std = np.std(numbers_of_leaving_from_2015_to_2020)
# The standardized ratios of company A, H, and N respectively are:
for i in numbers_of_leaving_from_2015_to_2020:
    standardized.append((i - mean) / std)

In [15]:
print('The table of before and after standardized is as following:')
standardized = pd.DataFrame(data=[numbers_of_leaving_from_2015_to_2020, standardized], columns=['A', 'H', 'N'], index=['Before', 'After'])
pd.options.display.float_format = '{:.2f}'.format

The table of before and after standardized is as following:


In [16]:
standardized

,A,H,N
Before,208.40,86.60,0.00
After,1.29,-0.14,-1.15


### Question 6: Calculate the # of job promotions annually for the 3 companies. Which firm promotes more of their employees and how did you define a “promotion”? 

At first, I think "title" can implies the promotion an employee receives. However, it just shows the current title. Then, I cannot find any other indicators of "promotions" other than "number_of_recommendations". In my opinion, the number_of_recommendations is directly propotional with promotions because it means how much credit the person gains from his / her friends, coworkers, and supervisors.
To find the job promotions annually, we have to divide the total number of recommendations by their working years

In [17]:
annual_proms_coms = []
annual_proms_coms.append(['Company A', com_a['number_of_recommendations'].sum() / com_a['existing_work_years'].sum()])
annual_proms_coms.append(['Company H', com_h['number_of_recommendations'].sum() / com_h['existing_work_years'].sum()])
annual_proms_coms.append(['Company N', com_n['number_of_recommendations'].sum() / com_n['existing_work_years'].sum()])
# I add all into a list and then sort them in accordance with the average number of promotions in descending order
annual_proms_coms.sort(key = lambda x: x[1], reverse=True)

In [18]:
print('The most promotions employees receive is from ' + str(annual_proms_coms[0][0]) + ' with the average being ' + str(annual_proms_coms[0][1]))

The most promotions employees receive is from Company A with the average being 2.566266257225433


### Question 7: From 2015 to 2020 annually, what was the average tenure of employees at each of the 3 firms?
#### a.	Please calculate in terms of avg # of years worked at the existing company

To simplify the problem, I assume the working year is calculated by subtracting the year of end_day and the year of start_day (I do not count specifically if it is a full year - 365 days or not).

In [19]:
avg_years = []
avg_years.append(['Company A', com_a['existing_work_years'].sum() / com_a['existing_work_years'].count()])
avg_years.append(['Company H', com_h['existing_work_years'].sum() / com_h['existing_work_years'].count()])
avg_years.append(['Company N', com_n['existing_work_years'].sum() / com_n['existing_work_years'].count()])

This is the list of companies and the respective average of existing working years


In [20]:
avg_years

[['Company A', 1.8555387967152672],
 ['Company H', 2.026609601786379],
 ['Company N', 2.453333950303086]]

#### b.	Also compute the total work experience for each employee for the final question below

For this problem, I'll show how many days each employee has worked since they started their first jobs and their ids

In [21]:
df[['id', 'total_working_days']]

,id,total_working_days
0,2ff517bd-3566-481e-a289-52ce4fb4ae2e,3803.00
1,78a0c7be-de6e-4af6-bab7-050a436e19fe,2434.00
2,78a0c7be-de6e-4af6-bab7-050a436e19fe,2434.00
3,78a0c7be-de6e-4af6-bab7-050a436e19fe,1764.00
4,78a0c7be-de6e-4af6-bab7-050a436e19fe,1764.00
...,...,...
32947,6c438838-359b-492a-8b64-7353a0a94462,4291.00
32948,6c438838-359b-492a-8b64-7353a0a94462,4688.00
32949,6c438838-359b-492a-8b64-7353a0a94462,4688.00
32950,6c438838-359b-492a-8b64-7353a0a94462,4688.00


### 8.	How would you score each of the firms’ culture from 1-100 using this data? Assume a score of 100 represents the highest quality and a score of 0 would be the lowest. 
#### a.	Hint: Use the aggregate employees’ tenure, total working experience, and education level (bachelor vs. masters vs. PhD) 


I will add all scores on 100 scale of each employee of each company, then divided by the number of employees of each company to get each company's score culture.
<br> There are three criteria to consider calculating scores of employees individually: employees’ tenure, total working experience, and education level
* Employees’ tenure: I'll put them into 5 bins
* Total working experience: I'll put them into 5 bins
* Education level: I'll put them into 7 bins
<br>So, there are totally 7*5*5 = 175 unique sets containing the three criteria.
<br>Then, I will count how many unique sets each company has divided by 175 and multiplied by 100 -> diversity score

In [22]:
def extract_degree(d):
    if 'postgraduate'.lower() in d or 'post graduate'.lower() in d:
         return 'PhD'
    elif 'Master' in d or "Master's" in d or "graduate".lower() in d or 'MBA'.lower() in d or 'ma'.lower() in d or 'graduate'.lower() in d or 'graduation'.lower() in d:
        return "Master"
    elif "Bachelor's" in d or 'BA'.lower() in d or 'BS'.lower() in d or 'BSC'.lower() in d or 'undergraduate'.lower() in d:
        return 'Bachelor'
    elif 'A-level'.lower() in d or 'as'.lower() in d:
        return 'Associate'
    elif 'high school'.lower() in d:
        return 'High school'
    elif 'certificate'.lower() in d or 'certified'.lower() in d or 'licence'.lower() in d or 'license'.lower() in d or 'diploma'.lower() in d:
        return 'Certificate'
    else:
        return "Other"

In [23]:
def binnings(a_list, number_of_bins=5):
    maximum = max(a_list)
    minimum = min(a_list)
    distance = (maximum - minimum) / number_of_bins
    binnings = []
    for i in range(number_of_bins):
        binnings.append(minimum + i*distance)
    return binnings

def assign_to_bin(score, binning_list):
    my_bin = 0
    for i in range(len(binning_list)):
        if score > binning_list[i]:
            my_bin = i + 1
        else:
            break
    return my_bin

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [25]:
com_a_score = com_a[['existing_work_years', 'total_working_days', 'degree']]
com_a_score.existing_work_years = com_a_score.existing_work_years.astype(float)
com_a_score.total_working_days = com_a_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_a_score['degree'].fillna("General Bachelor's", inplace=True) 
com_a_score['degree_type'] = com_a_score['degree'].apply(extract_degree)
com_a_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_a_score['existing_work_years']))) for x in com_a_score['existing_work_years'])
com_a_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_a_score['total_working_days']))) for x in com_a_score['total_working_days'])
com_a_score['degree_encoded'] = le.fit_transform(com_a_score['degree_type'])
com_a_diversity_code = list(zip(com_a_score.existing_work_years_encoded, com_a_score.total_working_days_encoded, com_a_score.degree_encoded))
com_a_diversity_score = 0
com_a_diversity_sets = set()
for i in com_a_diversity_code:
    if i not in com_a_diversity_sets:
        com_a_diversity_sets.add(i)
        com_a_diversity_score += 1
com_a_diversity_score = com_a_diversity_score * 100 / 175

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [26]:
com_h_score = com_h[['existing_work_years', 'total_working_days', 'degree']]
com_h_score.existing_work_years = com_h_score.existing_work_years.astype(float)
com_h_score.total_working_days = com_h_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_h_score['degree'].fillna("General Bachelor's", inplace=True) 
com_h_score['degree_type'] = com_h_score['degree'].apply(extract_degree)
com_h_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_h_score['existing_work_years']))) for x in com_h_score['existing_work_years'])
com_h_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_h_score['total_working_days']))) for x in com_h_score['total_working_days'])
com_h_score['degree_encoded'] = le.fit_transform(com_h_score['degree_type'])
com_h_diversity_code = list(zip(com_h_score.existing_work_years_encoded, com_h_score.total_working_days_encoded, com_h_score.degree_encoded))
com_h_diversity_score = 0
com_h_diversity_sets = set()
for i in com_h_diversity_code:
    if i not in com_h_diversity_sets:
        com_h_diversity_sets.add(i)
        com_h_diversity_score += 1
com_h_diversity_score = com_h_diversity_score * 100 / 175

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [27]:
com_n_score = com_n[['existing_work_years', 'total_working_days', 'degree']]
com_n_score.existing_work_years = com_n_score.existing_work_years.astype(float)
com_n_score.total_working_days = com_n_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_n_score['degree'].fillna("General Bachelor's", inplace=True) 
com_n_score['degree_type'] = com_n_score['degree'].apply(extract_degree)
com_n_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_n_score['existing_work_years']))) for x in com_n_score['existing_work_years'])
com_n_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_n_score['total_working_days']))) for x in com_n_score['total_working_days'])
com_n_score['degree_encoded'] = le.fit_transform(com_n_score['degree_type'])
com_n_diversity_code = list(zip(com_n_score.existing_work_years_encoded, com_n_score.total_working_days_encoded, com_n_score.degree_encoded))
com_n_diversity_score = 0
com_n_diversity_sets = set()
for i in com_n_diversity_code:
    if i not in com_n_diversity_sets:
        com_n_diversity_sets.add(i)
        com_n_diversity_score += 1
com_n_diversity_score = com_n_diversity_score * 100 / 175

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [28]:
com_a_diversity_score

41.714285714285715

In [29]:
com_h_diversity_score

13.714285714285714

In [30]:
com_n_diversity_score

24.571428571428573

If I have time, I can create functions to make the code look more professional

#### b.	What additional data would you like for each individual in order to help build a better culture ranking model? 

Gender, Race, Locust of control, Political affiliation, Born in the US?, First generation? Individualism or Collectivism? 